# 02. LLM Synthetic Data Generation

이 노트북은 LLM을 사용하여 합성 쿼리 데이터를 생성합니다.

## 전제조건
먼저 `01_neural_sparse_base_training.ipynb`를 실행하여 기본 데이터를 생성해야 합니다.

## 목표
- LLM 모델 로딩 (Qwen2.5-14B-Instruct BF16)
- 한국어 문서 기반 합성 쿼리 생성
- 동의어 확장 및 검증

## 출력 데이터
모든 데이터는 `dataset/llm_generated/` 디렉토리에 저장됩니다:
- `synthetic_qd_pairs.pkl`: LLM 생성 Query-Document 쌍
- `enhanced_synonyms.json`: LLM 검증 동의어 사전

## 다음 단계
이 노트북 실행 후 `03_llm_enhanced_training.ipynb`를 실행하세요.

In [13]:
# BF16 모델은 순수 PyTorch로 작동하므로 Triton 불필요
print("✅ Using BF16 model - no Triton configuration needed")

✅ Using BF16 model - no Triton configuration needed


In [14]:
# DatasetManager 초기화
from src.dataset_manager import DatasetManager

dm = DatasetManager(base_path="dataset")
print("✅ DatasetManager initialized")
print(f"📁 Base path: {dm.base_path.absolute()}")

✅ DatasetManager initialized
📁 Base path: /home/west/Documents/cursor-workspace/opensearch-neural-pre-train/dataset


In [15]:
# 필수 데이터 파일 확인
required_files = [
    ("base_model", "documents.json"),
    ("base_model", "bilingual_synonyms.json"),
]

if not dm.check_dependencies(required_files):
    raise RuntimeError(
        "Required data files not found. "
        "Please run 01_neural_sparse_base_training.ipynb first."
    )

print("\n✅ Ready to proceed with LLM data generation")

✅ All dependencies satisfied

✅ Ready to proceed with LLM data generation


In [16]:
# Notebook 1에서 생성된 데이터 로드
print("Loading data from notebook 1...\n")

documents = dm.load_json("documents.json", "base_model")
bilingual_dict = dm.load_json("bilingual_synonyms.json", "base_model")

print(f"\n✅ Loaded {len(documents)} documents")
print(f"✅ Loaded bilingual dictionary with {len(bilingual_dict)} entries")

Loading data from notebook 1...

✓ Loaded JSON: dataset/base_model/documents.json
✓ Loaded JSON: dataset/base_model/bilingual_synonyms.json

✅ Loaded 9996 documents
✅ Loaded bilingual dictionary with 32 entries


## 🆕 13. LLM 모델 로딩 및 초기화

이 섹션부터는 LLM 기반 확장 기능입니다:
- 합성 Query-Document 쌍 생성
- 한영 동의어 자동 검증
- 성능 개선

In [17]:
# Section 13: LLM Model Loading with Ollama

print("="*70)
print("🤖 섹션 13: LLM 모델 로딩 및 초기화 (Ollama)")
print("="*70)

from src.llm_loader import load_ollama_model

# Ollama 서버에서 Qwen3 모델 로딩
print("\n💡 참고: Ollama를 사용하여 로컬 모델을 로드합니다.")
print("   모델: qwen3:30b-a3b-thinking-2507-fp16")
print("   특징: 로컬 서버, 빠른 로딩, 효율적인 메모리 사용")
print("   요구사항: ollama serve 실행 중이어야 함")

llm_model, llm_tokenizer = load_ollama_model(
    model_name="qwen3:30b-a3b-thinking-2507-fp16",
)

print("\n✅ LLM 모델 로딩 완료!")
print("   모델을 사용하여 합성 데이터를 생성할 준비가 되었습니다.")
print("   예상 추론 속도: Ollama 최적화로 빠른 응답")

🤖 섹션 13: LLM 모델 로딩 및 초기화 (Ollama)

💡 참고: Ollama를 사용하여 로컬 모델을 로드합니다.
   모델: qwen3:30b-a3b-thinking-2507-fp16
   특징: 로컬 서버, 빠른 로딩, 효율적인 메모리 사용
   요구사항: ollama serve 실행 중이어야 함

📥 Loading Ollama Model
Model: qwen3:30b-a3b-thinking-2507-fp16
Host: http://localhost:11434
💡 Using Ollama (local server, zero setup)

1️⃣ Connecting to Ollama server...
✓ Connected to Ollama server
✓ Model 'qwen3:30b-a3b-thinking-2507-fp16' is available

2️⃣ Creating model wrapper...
✓ Model wrapper created

✅ Ollama Model Ready!
📊 Model uses Ollama server (no GPU memory tracked here)
    View Ollama logs for resource usage

✅ LLM 모델 로딩 완료!
   모델을 사용하여 합성 데이터를 생성할 준비가 되었습니다.
   예상 추론 속도: Ollama 최적화로 빠른 응답


## 🆕 14. LLM 기반 합성 Query-Document Pairs 생성

LLM을 사용하여 문서로부터 검색 쿼리를 역생성합니다.
이를 통해 학습 데이터를 대폭 확장할 수 있습니다.

In [18]:
print("\n" + "="*70)
print("📝 섹션 14: LLM 기반 합성 Query-Document Pairs 생성")
print("="*70)

from src.synthetic_data_generator import generate_synthetic_qd_pairs

# 테스트 모드: 처음 10개 문서로 시작
# 실제 운영 시 max_documents=1000으로 변경
print(f"\n📊 합성 데이터 생성 설정 (테스트 모드):")
print(f"   - 사용할 문서: 10개 (테스트)")
print(f"   - 문서당 쿼리: 3개")
print(f"   - 예상 생성 pairs: ~30개")
print(f"   - 모델: Qwen3-30B via Ollama")
print(f"   - 예상 소요 시간: ~5-10분")
print(f"\n💡 테스트 완료 후 max_documents를 늘려서 실행하세요.")

synthetic_pairs = generate_synthetic_qd_pairs(
    documents=documents[:1000],  # 테스트: 10개 문서만
    llm_model=llm_model,
    llm_tokenizer=llm_tokenizer,
    num_queries_per_doc=3,
    batch_size=2,
    max_documents=1000,
    enable_filtering=True,
    verbose=True,  # 상세 로그 활성화
)

print(f"\n✅ 합성 데이터 생성 완료: {len(synthetic_pairs):,}개 pairs")

# 샘플 출력
print("\n📋 생성된 합성 데이터 샘플 (처음 5개):")
print("="*70)
for i, (query, doc, relevance) in enumerate(synthetic_pairs[:5], 1):
    print(f"\n{i}. Query: {query}")
    print(f"   Document: {doc[:100]}...")
    print(f"   Relevance: {relevance}")
print("="*70)


📝 섹션 14: LLM 기반 합성 Query-Document Pairs 생성

📊 합성 데이터 생성 설정 (테스트 모드):
   - 사용할 문서: 10개 (테스트)
   - 문서당 쿼리: 3개
   - 예상 생성 pairs: ~30개
   - 모델: Qwen3-30B via Ollama
   - 예상 소요 시간: ~5-10분

💡 테스트 완료 후 max_documents를 늘려서 실행하세요.

📝 Generating Synthetic Query-Document Pairs
Start time: 2025-11-16 18:31:32
Documents: 1000
Queries per doc: 3
Expected total queries: 3000
Quality filtering: ON
Verbose logging: ON


Generating queries:   0%|                                                  | 0/1000 [00:00<?, ?it/s]


📊 Progress Report - Document 1/1000
⏱️  Elapsed: 0.0m | ETA: 0.0m
✅ Generated: 0 pairs
❌ Failed/Filtered: 0
⚡ Avg time/doc: 0.00s

📄 Doc 1: Over 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID Boosters...
   🤖 Calling LLM...


Generating queries:   0%|                                        | 1/1000 [00:18<5:07:32, 18.47s/it]

   ✅ LLM responded in 18.47s
   📝 Generated 3 queries:
      1. Okay, let's see. The user wants me to generate three short, specific search queries (5-15 words each) based on the given document. The document title is "Over 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID Boosters".
      2. First, I need to understand the key points here. The main elements are: over 4 million Americans, rolling up sleeves (which means getting vaccinated), Omicron-targeted, and COVID boosters.
      3. So the goal is to create search terms that someone might use to find this document. They should be concise, 5-15 words. Let me break it down.
      ⚠️  Filtered: Okay, let's see. The user wants me to generate thr...
      ⚠️  Filtered: First, I need to understand the key points here. T...
      ⚠️  Filtered: So the goal is to create search terms that someone...
   ➕ Added 0 pairs
   ⏱️  Total time for this doc: 18.47s

📄 Doc 2: American Airlines Flyer Charged, Banned For Life After Punching 

...

   ✅ LLM responded in 4.82s
   📝 Generated 3 queries:
      1. Okay, let's tackle this query. The user wants three short and specific search queries (5-15 words each) based on the given document about Syria's Assad visiting the UAE.
      2. First, I need to understand the key points of the document. The main elements are: Syria's Assad, UAE visit, first trip to an Arab country since the war.
      3. So, the user is looking for search terms that someone might use to find this document. Let me break down the important parts.
      ⚠️  Filtered: Okay, let's tackle this query. The user wants thre...
      ⚠️  Filtered: First, I need to understand the key points of the ...
      ⚠️  Filtered: So, the user is looking for search terms that some...
   ➕ Added 0 pairs
   ⏱️  Total time for this doc: 4.82s

📄 Doc 999: Strong Japan Earthquake Kills 4, Injures 107...
   🤖 Calling LLM...


Generating queries: 100%|█████████████████████████████████████▉| 999/1000 [1:21:02<00:04,  4.86s/it]

   ✅ LLM responded in 4.89s
   📝 Generated 3 queries:
      1. Okay, I need to create three short and specific search queries for the given document about a Japan earthquake. Let me start by understanding the document.
      2. The document says: "Strong Japan Earthquake Kills 4, Injures 107". So the key points are Japan, strong earthquake, 4 deaths, 107 injured.
      3. The user wants search queries that are 5-15 words each, concise and specific. Let me think about what someone might search for to find this document.
      ⚠️  Filtered: Okay, I need to create three short and specific se...
      ⚠️  Filtered: The document says: "Strong Japan Earthquake Kills ...
      ⚠️  Filtered: The user wants search queries that are 5-15 words ...
   ➕ Added 0 pairs
   ⏱️  Total time for this doc: 4.89s

📄 Doc 1000: 9 Dead, 2 Injured In Fiery Crash Involving University Of The Southwest Golf Team...
   🤖 Calling LLM...


Generating queries: 100%|█████████████████████████████████████| 1000/1000 [1:21:07<00:00,  4.87s/it]

   ✅ LLM responded in 4.87s
   📝 Generated 3 queries:
      1. Okay, let's see. The user wants me to generate three short, specific search queries (5-15 words each) based on the given document. The document is about a fiery crash involving the University of the Southwest golf teams, with 9 dead and 2 injured.
      2. First, I need to make sure the queries are concise and directly related to the main points. The key elements here are the number of fatalities (9 dead), the injured (2), the cause (fiery crash), and the involved parties (University of the Southwest golf teams).
      3. Let me break it down. The main event is the crash, so "fiery crash" should be in there. The university name is important, so "University of the Southwest
      ⚠️  Filtered: Okay, let's see. The user wants me to generate thr...
      ⚠️  Filtered: First, I need to make sure the queries are concise...
      ⚠️  Filtered: Let me break it down. The main event is the crash,...
   ➕ Added 0 pairs
   ⏱️  Total t

## 🆕 15. LLM 기반 한영 동의어 검증 및 확장

기존 임베딩 기반으로 발견한 동의어를 LLM으로 검증하여 정확도를 높입니다.

In [19]:
print("\n" + "="*70)
print("🌐 섹션 15: LLM 기반 한영 동의어 검증 및 확장")
print("="*70)

from src.cross_lingual_synonyms import enhance_bilingual_dict_with_llm

# 기존 임베딩 기반 동의어를 LLM으로 검증
print(f"\n📊 동의어 검증 설정:")
print(f"   - 기존 동의어 사전: {len(bilingual_dict):,}개 항목")
print(f"   - 검증할 항목: 상위 100개")
print(f"   - 검증 임계값: 0.8")
print(f"   - 예상 소요 시간: ~5-10분")

enhanced_bilingual_dict = enhance_bilingual_dict_with_llm(
    initial_dict=bilingual_dict,  # Cell 14에서 생성된 기존 사전
    llm_model=llm_model,
    llm_tokenizer=llm_tokenizer,
    verification_threshold=0.8,
    max_verify=100,  # 상위 100개만 검증 (시간 절약)
)

print(f"\n✅ LLM 검증 완료!")
print(f"   기존 사전: {len(bilingual_dict):,}개")
print(f"   검증 후: {len(enhanced_bilingual_dict):,}개")

# 검증 결과 샘플 출력
print("\n📋 검증된 동의어 샘플:")
print("="*70)
sample_count = 0
for korean, english_list in list(enhanced_bilingual_dict.items())[:10]:
    sample_count += 1
    print(f"{sample_count}. {korean} ↔ {', '.join(english_list)}")
print("="*70)

# 업데이트된 동의어 사전을 bilingual_dict에 반영
bilingual_dict = enhanced_bilingual_dict
print(f"\n✓ 동의어 사전 업데이트 완료")


🌐 섹션 15: LLM 기반 한영 동의어 검증 및 확장

📊 동의어 검증 설정:
   - 기존 동의어 사전: 32개 항목
   - 검증할 항목: 상위 100개
   - 검증 임계값: 0.8
   - 예상 소요 시간: ~5-10분

🤖 LLM-based Bilingual Synonym Verification
Initial dictionary size: 32
Max pairs to verify: 100

  Verifying 32 terms...


LLM verification: 100%|██████████| 32/32 [03:58<00:00,  7.45s/it]


✅ LLM Verification Complete
Verified pairs: 3
Rejected pairs: 65
Final dictionary size: 32

✅ LLM 검증 완료!
   기존 사전: 32개
   검증 후: 32개

📋 검증된 동의어 샘플:
1. 신경망 ↔ net
2. 문서 ↔ doc
3. 분석 ↔ analysis
4. 모델 ↔ model, Model
5. 학습 ↔ learning, training, train
6. 검색 ↔ search, retrieval, retrieve
7. 데이터 ↔ data, Data
8. 알고리즘 ↔ algorithm, Algorithm
9. 벡터 ↔ vector, Vector
10. 임베딩 ↔ embedding, Embedding

✓ 동의어 사전 업데이트 완료


---

# 데이터 저장 (Notebook 간 공유)

LLM으로 생성된 데이터를 `dataset/llm_generated/` 디렉토리에 저장합니다.
다음 노트북에서 이 데이터를 로드하여 enhanced 모델을 학습할 수 있습니다.

In [20]:
# 1. LLM 생성 Query-Document 쌍 저장
dm.save_pickle(
    synthetic_pairs,
    "synthetic_qd_pairs.pkl",
    "llm_generated"
)

✓ Saved Pickle: dataset/llm_generated/synthetic_qd_pairs.pkl


PosixPath('dataset/llm_generated/synthetic_qd_pairs.pkl')

In [21]:
# 2. LLM 검증 동의어 사전 저장
dm.save_json(
    enhanced_bilingual_dict,
    "enhanced_synonyms.json",
    "llm_generated"
)

✓ Saved JSON: dataset/llm_generated/enhanced_synonyms.json


PosixPath('dataset/llm_generated/enhanced_synonyms.json')

In [22]:
# 저장된 데이터 요약
dm.print_summary()

📊 Dataset Summary
Base path: /home/west/Documents/cursor-workspace/opensearch-neural-pre-train/dataset
Total datasets: 9

Datasets by directory:

  📁 base_model/
     - documents.json                           (           json,    0.7 MB)
     - idf_statistics.pkl                       (         pickle,    0.3 MB)
     - qd_pairs_base.pkl                        (         pickle,   54.9 MB)
     - bilingual_synonyms.json                  (           json,    0.0 MB)
     - neural_sparse_v1_model                   (  pytorch_model,  423.1 MB)
     Total:                                                     479.1 MB

  📁 llm_generated/
     - synthetic_qd_pairs.pkl                   (         pickle,    0.0 MB)
     - enhanced_synonyms.json                   (           json,    0.0 MB)
     - data_analysis.json                       (           json,    0.0 MB)
     - combined_qd_pairs.pkl                    (         pickle,   29.2 MB)
     Total:                                         

---

## ✅ Notebook 2 완료

모든 LLM 생성 데이터가 `dataset/llm_generated/`에 저장되었습니다.

### 다음 단계
이제 `03_llm_enhanced_training.ipynb`를 실행하여 enhanced 모델을 학습하고 성능을 비교하세요.